<h1>서울특별시 장애인 관련 사회복지시설목록 지오코딩</h1>
<h3>서울특별시 장애인 관련 사회복지시설 목록에 QGIS 점좌표나 X,Y좌표가 없어서 분석이 불가함.</h3>
<h3>-> 이를 해결하기 위해 주소를 좌표로 변환하는 지오코딩 실시.</h3>

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import requests
import googlemaps
import urllib
from xml.etree.ElementTree import parse
from bs4 import BeautifulSoup

In [ ]:
#구글 Maps API KEY
api_key = ''

In [ ]:
#서울시 장애인 관련 사회복지시설 목록 CSV파일을 불러옴.
dis = pd.read_csv('서울특별시 장애인 관련 사회복지시설목록.csv', encoding = 'euc-kr')

In [ ]:
#서울시 장애인 관련 사회복지 시설 목록에서 주소와 시설명을 불러옴
#-> 이것을 구글 지오코딩을 이용해 lon, lat 좌표로 변환함.
lon = []
lat = []
e_l = []
for i in range(len(dis)):
    a = re.sub(r'\([^)]*\)','',dis.iloc[i,9])
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={a}&key={api_key}&language=ko'
    res = requests.get(url)
    lon.append(res.json()['results'][0]['geometry']['location']['lng'])
    lat.append(res.json()['results'][0]['geometry']['location']['lat'])
    e_l.append(a)
        
    print(i+1/len(dis.시설주소))

In [ ]:
#구해온 lon, lat 좌표를 원래 DF에 추가한 후 저장.
dis['lon'] = lon
dis['lat'] = lat

dis.to_csv('서울특별시장애인거주시설_좌표추가_구글지오코딩.csv', index = False, encoding = 'UTF-8')

In [ ]:
#주민자치센터 데이터를 불러옴.
se = pd.read_csv('서울시 주민자치센터 정보.csv')

In [ ]:
#데이터 확인
dis.info()

In [ ]:
#서울시 주민자치센터 목록에서 도로명 주소를 불러옴.
#-> 이것을 구글 지오코딩을 이용해 lon, lat 좌표로 변환함.
lon = []
lat = []
for idx, a in enumerate(se['도로명 주소']):
    a = re.sub(r'\([^)]*\)','',a)
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={a}&key={api_key}&language=ko'
    res = requests.get(url)
    lon.append(res.json()['results'][0]['geometry']['location']['lng'])
    lat.append(res.json()['results'][0]['geometry']['location']['lat'])
    
    print((idx + 1)/len(se['도로명 주소']))

In [ ]:
se['lon'] = lon
se['lat'] = lat

In [ ]:
se.to_csv('서울특별시_주민자치센터_좌표추가_구글지오코딩.csv', index = False)

<h1>각각 장애인관련 사회복지 시설들 Clustering</h1>
<h3>사고가 자주 발생하는 위험한 집단을 찾아 장애인구역 우선 설치를 권장하기 위해 Clustering 실시. Clustering의 성능 향상을 위해 우선 차원축소를 실시한 뒤 클러스터링을 실시할 예정.</h3>

In [ ]:
import umap 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.metrics import silhouette_samples, silhouette_score
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn_extra.cluster import KMedoids
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from hyperopt.pyll import scope

In [ ]:
#Clustering을 위해 사용할 시설별 정보 파일을 불러옴.
dis_h = pd.read_csv('/Users/user/Desktop/팀프/분석용 데이터 최종_과속방지턱제거_불량산책로추가_진짜최종.csv', encoding = 'EUC-KR', index_col=0)

In [ ]:
#사용할 컬럼을 솎아내기위해 컬럼이 어떤것이 있나 확인.
dis_h

In [ ]:
#Clustering을 할 수 있도록 사용할 컬럼들만 추려냄.
dis_h_2 = dis_h.loc[:, [x for x in dis_h.columns if x != 'total']]

In [ ]:
dis_h_2.tail()

In [ ]:
#추후 편리하게 사용하기위해 비교할 사고횟수를 따로 빼둠.
acc_c = dis_h.total

In [ ]:
#특정 컬럼때문에 공간변환이 틀어지는 것을 예방하기 위해 Standard Scaler를 이용
scaler = StandardScaler()
dis_h_2_scaled = scaler.fit_transform(dis_h_2)
dis_h_2_scaled

In [ ]:
#Scaled된 Data를 일단 dis_h_2_scaled.shape[1]( = 11 )차원으로 공간변환함. 누적기여율을 확인하고 사용할 예정. 
pca = PCA(n_components=dis_h_2_scaled.shape[1])
dis_h_pca_t = pca.fit_transform(dis_h_2_scaled)
result = pd.DataFrame({'설명가능한 분산 비율(고윳값)':pca.explained_variance_,
             '기여율':pca.explained_variance_ratio_},
            index=np.array([f"pca{num+1}" for num in range(dis_h_2_scaled.shape[1])]))
result['누적기여율'] = result['기여율'].cumsum()
result

<h4>PCA 8에서  누적 기여율이 80%를 넘고 고유값도 0.7에 매우 근사함. -> 8차원까지 공간변환 할것임.</h4>

In [ ]:
pca = PCA(n_components=8)
dis_h_pca = pca.fit_transform(dis_h_2_scaled)

In [ ]:
# 7차원의 PCA변환된 데이터로 K-Means를 해봄.

sse = []
silhouette_coefficients = []
for i in range(2, 11):
    kmeans = KMeans(init="random",
                n_clusters=i,
                n_init=10,
                max_iter=300,
                random_state=42
    )
    kmeans.fit(dis_h_pca)
    sse.append(kmeans.inertia_)
    score = silhouette_score(dis_h_pca, kmeans.labels_)
    silhouette_coefficients.append(score)
    
    print(f'{i}  kmeans inertia : {kmeans.inertia_}')
    print(f'{i}  kmeans score : {score}')

plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), sse)
plt.xticks(range(2, 11))
plt.xlabel("클러스터 수")
plt.ylabel("SSE")
plt.show()

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("silhouette_score")
plt.show()

In [ ]:
#6에서 변곡점과 유사한형태가 보임 -> 6에 실루엣그래프 그려봄.
fig, ax = plt.subplots(figsize=(8,7))
km = KMeans(n_clusters=7, init='random',max_iter=500, random_state=42)
ax.set_facecolor((1.0, 1.0,1.0))
plt.rcParams['figure.facecolor'] = 'white'
visualizer = SilhouetteVisualizer(km, colors='yellowbrick')
visualizer.fit(dis_h_pca)


성능이 매우 좋지않음

<h4>PCA 한 후 Clustering이 잘 안됨 -> 분산을이용하는 PCA보다 매니폴드 가정을 이용하는 UMAP이용</h4>

UMAP은 n_neighbor, min_dist라는 hyperparameter가 있으며, 이에 따라 차원축소 결과가 상이하게달라짐.  
UMAP을 Manually하게 search하고 거기에 K-Means와 K-Medoids 전부 해보면서 k정한다는건 사실상힘듬 -> 베이지안최적화를 이용하는 hyperopt를 사용해 hyperparameter를 search할 것임.  
search space :  
    1. K : 해석 용이하게 통상적으로 많이 쓰는 3-7개로 돌려볼것임. K가 적을수록 변동계수가 적어지는 경향이 있기에 K당 한번씩 찾아볼것.  
    2. Model : K-means, K-medoids  
    3. n_neighbor : 작을수록 locality 강조. 차원보다 높게 설정하는것이 권장되기에 4-50으로 넣어봄.  
    4. min_dist : point를 얼마나 촘촘하게 묶을지 결정.  0.01부터 0.9까지 넣어볼것임.
    
최적화할 Loss 정의 :  
실루엣 그래프를 판단할 때 중요한것이 군집별 실루엣계수가 얼마나 일정한가임. -> 군집별 실루엣 계수들의 변동계수를 Loss로 지정 후 이것을 최소화하는 방향으로 최적화 실행  

그렇게 복잡한 공간도아니고 시각화하는것도 중요한 요소기에 3차원으로 변환할것임.  

각각 K에 대한 최적의 결과에서 군집별 사고 횟수 차이가 가장 큰 것을 선택할것.

In [ ]:
'''
K가 n일때 최적화된 UMAP, MODEL 확인.
'''

def search_space():
    space = {'model' : hp.choice('model', ('kmeans','kmedoids')),
             'n_n': scope.int(hp.quniform('n_n', 4, 50, q=1)),
             'm_d': hp.uniform('m_d', 0.01, 0.9)}
    return space

def return_cv(sil_avg, sil_values, cluster_labels):
    return {'loss' : np.std(pd.DataFrame({'label' : cluster_labels, 'sil_values' : sil_values}).groupby(['label']).mean().sil_values) / sil_avg, 'status': STATUS_OK}

def obj_fnc(params):
    dis_h_um = umap.UMAP(n_neighbors=params.get('n_n'),
                 min_dist = params.get('m_d'),
                 n_components = 3,
                 random_state = 42).fit_transform(dis_h_2_scaled)
    
    if params.get('model') == 'kmeans':
        clusterer = KMeans(init = 'random', n_clusters = n, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
    if params.get('model') == 'kmedoids':
        clusterer = KMedoids(init="random", n_clusters = n, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
hypopt_trials_3 = Trials()
best_params = fmin(obj_fnc, search_space(), algo=tpe.suggest, max_evals=1000, trials= hypopt_trials_3, rstate = np.random.RandomState(42) )

In [ ]:
'''
K가 3일때 최적화된 UMAP, MODEL 확인.
'''

def search_space():
    space = {'model' : hp.choice('model', ('kmeans','kmedoids')),
             'n_n': scope.int(hp.quniform('n_n', 4, 50, q=1)),
             'm_d': hp.uniform('m_d', 0.01, 0.9)}
    return space

def return_cv(sil_avg, sil_values, cluster_labels):
    return {'loss' : np.std(pd.DataFrame({'label' : cluster_labels, 'sil_values' : sil_values}).groupby(['label']).mean().sil_values) / sil_avg, 'status': STATUS_OK}

def obj_fnc(params):
    dis_h_um = umap.UMAP(n_neighbors=params.get('n_n'),
                 min_dist = params.get('m_d'),
                 n_components = 3,
                 random_state = 42).fit_transform(dis_h_2_scaled)
    
    if params.get('model') == 'kmeans':
        clusterer = KMeans(init = 'random', n_clusters = 3, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
    if params.get('model') == 'kmedoids':
        clusterer = KMedoids(init="random", n_clusters = 3, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
hypopt_trials_3 = Trials()
best_params = fmin(obj_fnc, search_space(), algo=tpe.suggest, max_evals=1000, trials= hypopt_trials_3, rstate = np.random.RandomState(42) )

In [ ]:
n_n = int(hypopt_trials_3.best_trial['misc']['vals']['n_n'][0])
m_dist = hypopt_trials_3.best_trial['misc']['vals']['m_d'][0]
r_m_dist = round(m_dist, 3)
dis_h_um = umap.UMAP(n_neighbors=n_n,
                     min_dist = m_dist,
                     n_components = 3,
                     random_state = 42).fit_transform(dis_h_2_scaled)
if hypopt_trials_3.best_trial['misc']['vals']['model'][0] == 0:
    clusterer = KMeans(init = 'random', n_clusters = 3, max_iter=500, random_state=42)
    cluster_labels = clusterer.fit_predict(dis_h_um)
    model_name = 'K-Means'
if hypopt_trials_3.best_trial['misc']['vals']['model'][0] == 1:
    clusterer = KMedoids(init = 'random', n_clusters = 3, max_iter=500, random_state=42)
    cluster_labels = clusterer.fit_predict(dis_h_um)
    model_name = 'K-Medoids'
    
fig = plt.figure(figsize = (20, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(dis_h_um[:,0], dis_h_um[:,1], dis_h_um[:,2], c = [sns.color_palette('bright')[x] for x in cluster_labels], s = 50)
ax.set_title(f'K가 3일때 군집 결과 \n 사용모델 : {model_name}, n_neighbor : {n_n}, min_dist : {r_m_dist}')

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
name = 'K-Means' if hypopt_trials_3.best_trial['misc']['vals']['model'][0] == 0 else 'K-Medoids'
name
fig.suptitle(f'{name} 실루엣 계수 그래프: K가 3일때')

visualizer = SilhouetteVisualizer(clusterer, colors='yellowbrick')
visualizer.fit(dis_h_um)

In [ ]:
sns.barplot(data = pd.DataFrame({'acc' : acc_c, 'cl_label' : cluster_labels}).groupby(['cl_label']).mean().reset_index(), x = 'cl_label', y = 'acc').set_title('군집 별 평균 사고 횟수')

In [ ]:
sns.barplot(data = pd.DataFrame(pd.DataFrame({'acc' : acc_c, 'cl_label' : cluster_labels}).cl_label.value_counts()).reset_index(), x = 'index', y = 'cl_label').set_title('군집 별 개체 갯수')

In [ ]:
'''
K가 4일때 최적화된 UMAP, MODEL 확인.
'''

def search_space():
    space = {'model' : hp.choice('model', ('kmeans','kmedoids')),
             'n_n': scope.int(hp.quniform('n_n', 4, 50, q=1)),
             'm_d': hp.uniform('m_d', 0.01, 0.9)}
    return space

def return_cv(sil_avg, sil_values, cluster_labels):
    return {'loss' : np.std(pd.DataFrame({'label' : cluster_labels, 'sil_values' : sil_values}).groupby(['label']).mean().sil_values) / sil_avg, 'status': STATUS_OK}

def obj_fnc(params):
    dis_h_um = umap.UMAP(n_neighbors=params.get('n_n'),
                 min_dist = params.get('m_d'),
                 n_components = 3,
                 random_state = 42).fit_transform(dis_h_2_scaled)
    
    if params.get('model') == 'kmeans':
        clusterer = KMeans(init = 'random', n_clusters = 4, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
    if params.get('model') == 'kmedoids':
        clusterer = KMedoids(init="random", n_clusters = 4, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
hypopt_trials_4 = Trials()
best_params = fmin(obj_fnc, search_space(), algo=tpe.suggest, max_evals=1000, trials= hypopt_trials_4, rstate = np.random.RandomState(42) )

In [ ]:
n_n = int(hypopt_trials_4.best_trial['misc']['vals']['n_n'][0])
m_dist = hypopt_trials_4.best_trial['misc']['vals']['m_d'][0]
r_m_dist = round(m_dist, 3)
dis_h_um = umap.UMAP(n_neighbors=n_n,
                     min_dist = m_dist,
                     n_components = 3,
                     random_state = 42).fit_transform(dis_h_2_scaled)
if hypopt_trials_4.best_trial['misc']['vals']['model'][0] == 0:
    clusterer = KMeans(init = 'random', n_clusters = 4, max_iter=500, random_state=42)
    cluster_labels = clusterer.fit_predict(dis_h_um)
    model_name = 'K-Means'
if hypopt_trials_4.best_trial['misc']['vals']['model'][0] == 1:
    clusterer = KMedoids(init = 'random', n_clusters = 4, max_iter=500, random_state=42)
    cluster_labels = clusterer.fit_predict(dis_h_um)
    model_name = 'K-Medoids'
    
fig = plt.figure(figsize = (20, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(dis_h_um[:,0], dis_h_um[:,1], dis_h_um[:,2], c = [sns.color_palette('bright')[x] for x in cluster_labels], s = 50)
ax.set_title(f'K가 4일때 군집 결과 \n 사용모델 : {model_name}, n_neighbor : {n_n}, min_dist : {r_m_dist}')

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
name = 'K-Means' if hypopt_trials_4.best_trial['misc']['vals']['model'][0] == 0 else 'K-Medoids'
name
fig.suptitle(f'{name} 실루엣 계수 그래프: K가 4일때')

visualizer = SilhouetteVisualizer(clusterer, colors='yellowbrick')
visualizer.fit(dis_h_um)

In [ ]:
sns.barplot(data = pd.DataFrame({'acc' : acc_c, 'cl_label' : cluster_labels}).groupby(['cl_label']).mean().reset_index(), x = 'cl_label', y = 'acc').set_title(f'군집 별 평균 사고 횟수')

In [ ]:
sns.barplot(data = pd.DataFrame(pd.DataFrame({'acc' : acc_c, 'cl_label' : cluster_labels}).cl_label.value_counts()).reset_index(), x = 'index', y = 'cl_label').set_title('군집 별 개체 갯수')

In [ ]:
'''
K가 5일때 최적화된 UMAP, MODEL 확인.
'''

def search_space():
    space = {'model' : hp.choice('model', ('kmeans','kmedoids')),
             'n_n': scope.int(hp.quniform('n_n', 4, 50, q=1)),
             'm_d': hp.uniform('m_d', 0.01, 0.9)}
    return space

def return_cv(sil_avg, sil_values, cluster_labels):
    return {'loss' : np.std(pd.DataFrame({'label' : cluster_labels, 'sil_values' : sil_values}).groupby(['label']).mean().sil_values) / sil_avg, 'status': STATUS_OK}

def obj_fnc(params):
    dis_h_um = umap.UMAP(n_neighbors=params.get('n_n'),
                 min_dist = params.get('m_d'),
                 n_components = 3,
                 random_state = 42).fit_transform(dis_h_2_scaled)
    
    if params.get('model') == 'kmeans':
        clusterer = KMeans(init = 'random', n_clusters = 5, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
    if params.get('model') == 'kmedoids':
        clusterer = KMedoids(init="random", n_clusters = 5, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
hypopt_trials_5 = Trials()
best_params = fmin(obj_fnc, search_space(), algo=tpe.suggest, max_evals=1000, trials= hypopt_trials_5, rstate = np.random.RandomState(42) )

In [ ]:
n_n = int(hypopt_trials_5.best_trial['misc']['vals']['n_n'][0])
m_dist = hypopt_trials_5.best_trial['misc']['vals']['m_d'][0]
r_m_dist = round(m_dist, 3)
dis_h_um = umap.UMAP(n_neighbors=n_n,
                     min_dist = m_dist,
                     n_components = 3,
                     random_state = 42).fit_transform(dis_h_2_scaled)
if hypopt_trials_5.best_trial['misc']['vals']['model'][0] == 0:
    clusterer = KMeans(init = 'random', n_clusters = 5, max_iter=500, random_state=42)
    cluster_labels = clusterer.fit_predict(dis_h_um)
    model_name = 'K-Means'
if hypopt_trials_5.best_trial['misc']['vals']['model'][0] == 1:
    clusterer = KMedoids(init = 'random', n_clusters = 5, max_iter=500, random_state=42)
    cluster_labels = clusterer.fit_predict(dis_h_um)
    model_name = 'K-Medoids'
    
fig = plt.figure(figsize = (20, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(dis_h_um[:,0], dis_h_um[:,1], dis_h_um[:,2], c = [sns.color_palette('bright')[x] for x in cluster_labels], s = 50)
ax.set_title(f'K가 5일때 군집 결과 \n 사용모델 : {model_name}, n_neighbor : {n_n}, min_dist : {r_m_dist}')

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
name = 'K-Means' if hypopt_trials_5.best_trial['misc']['vals']['model'][0] == 0 else 'K-Medoids'
name
fig.suptitle(f'{name} 실루엣 계수 그래프: K가 5일때')

visualizer = SilhouetteVisualizer(clusterer, colors='yellowbrick')
visualizer.fit(dis_h_um)

In [ ]:
sns.barplot(data = pd.DataFrame({'acc' : acc_c, 'cl_label' : cluster_labels}).groupby(['cl_label']).mean().reset_index(), x = 'cl_label', y = 'acc').set_title(f'군집 별 평균 사고 횟수')

In [ ]:
sns.barplot(data = pd.DataFrame(pd.DataFrame({'acc' : acc_c, 'cl_label' : cluster_labels}).cl_label.value_counts()).reset_index(), x = 'index', y = 'cl_label').set_title('군집 별 개체 갯수')

In [ ]:
'''
K가 6일때 최적화된 UMAP, MODEL 확인.
'''

def search_space():
    space = {'model' : hp.choice('model', ('kmeans','kmedoids')),
             'n_n': scope.int(hp.quniform('n_n', 4, 50, q=1)),
             'm_d': hp.uniform('m_d', 0.01, 0.9)}
    return space

def return_cv(sil_avg, sil_values, cluster_labels):
    return {'loss' : np.std(pd.DataFrame({'label' : cluster_labels, 'sil_values' : sil_values}).groupby(['label']).mean().sil_values) / sil_avg, 'status': STATUS_OK}

def obj_fnc(params):
    dis_h_um = umap.UMAP(n_neighbors=params.get('n_n'),
                 min_dist = params.get('m_d'),
                 n_components = 3,
                 random_state = 42).fit_transform(dis_h_2_scaled)
    
    if params.get('model') == 'kmeans':
        clusterer = KMeans(init = 'random', n_clusters = 6, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
    if params.get('model') == 'kmedoids':
        clusterer = KMedoids(init="random", n_clusters = 6, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
hypopt_trials_6 = Trials()
best_params = fmin(obj_fnc, search_space(), algo=tpe.suggest, max_evals=1000, trials= hypopt_trials_6, rstate = np.random.RandomState(42) )

In [ ]:
n_n = int(hypopt_trials_6.best_trial['misc']['vals']['n_n'][0])
m_dist = hypopt_trials_6.best_trial['misc']['vals']['m_d'][0]
r_m_dist = round(m_dist, 3)
dis_h_um = umap.UMAP(n_neighbors=n_n,
                     min_dist = m_dist,
                     n_components = 3,
                     random_state = 42).fit_transform(dis_h_2_scaled)
if hypopt_trials_6.best_trial['misc']['vals']['model'][0] == 0:
    clusterer = KMeans(init = 'random', n_clusters = 6, max_iter=500, random_state=42)
    cluster_labels = clusterer.fit_predict(dis_h_um)
    model_name = 'K-Means'
if hypopt_trials_6.best_trial['misc']['vals']['model'][0] == 1:
    clusterer = KMedoids(init = 'random', n_clusters = 6, max_iter=500, random_state=42)
    cluster_labels = clusterer.fit_predict(dis_h_um)
    model_name = 'K-Medoids'
    
fig = plt.figure(figsize = (20, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(dis_h_um[:,0], dis_h_um[:,1], dis_h_um[:,2], c = [sns.color_palette('bright')[x] for x in cluster_labels], s = 50)
ax.set_title(f'K가 6일때 군집 결과 \n 사용모델 : {model_name}, n_neighbor : {n_n}, min_dist : {r_m_dist}')

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
name = 'K-Means' if hypopt_trials_6.best_trial['misc']['vals']['model'][0] == 0 else 'K-Medoids'
name
fig.suptitle(f'{name} 실루엣 계수 그래프: K가 6일때')

visualizer = SilhouetteVisualizer(clusterer, colors='yellowbrick')
visualizer.fit(dis_h_um)

In [ ]:
sns.barplot(data = pd.DataFrame({'acc' : acc_c, 'cl_label' : cluster_labels}).groupby(['cl_label']).mean().reset_index(), x = 'cl_label', y = 'acc').set_title(f'군집 별 평균 사고 횟수')

In [ ]:
sns.barplot(data = pd.DataFrame(pd.DataFrame({'acc' : acc_c, 'cl_label' : cluster_labels}).cl_label.value_counts()).reset_index(), x = 'index', y = 'cl_label').set_title('군집 별 개체 갯수')

In [ ]:
'''
K가 7일때 최적화된 UMAP, MODEL 확인.
'''

def search_space():
    space = {'model' : hp.choice('model', ('kmeans','kmedoids')),
             'n_n': scope.int(hp.quniform('n_n', 4, 50, q=1)),
             'm_d': hp.uniform('m_d', 0.01, 0.9)}
    return space

def return_cv(sil_avg, sil_values, cluster_labels):
    return {'loss' : np.std(pd.DataFrame({'label' : cluster_labels, 'sil_values' : sil_values}).groupby(['label']).mean().sil_values) / sil_avg, 'status': STATUS_OK}

def obj_fnc(params):
    dis_h_um = umap.UMAP(n_neighbors=params.get('n_n'),
                 min_dist = params.get('m_d'),
                 n_components = 3,
                 random_state = 42).fit_transform(dis_h_2_scaled)
    
    if params.get('model') == 'kmeans':
        clusterer = KMeans(init = 'random', n_clusters = 7, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
    if params.get('model') == 'kmedoids':
        clusterer = KMedoids(init="random", n_clusters = 7, max_iter=500, random_state=42)
        cluster_labels = clusterer.fit_predict(dis_h_um)
        sil_avg = silhouette_score(dis_h_um, cluster_labels)
        sil_values = silhouette_samples(dis_h_um, cluster_labels)
        return(return_cv(sil_avg, sil_values, cluster_labels))
    
hypopt_trials_7 = Trials()
best_params = fmin(obj_fnc, search_space(), algo=tpe.suggest, max_evals=1000, trials= hypopt_trials_7, rstate = np.random.RandomState(42) )

In [ ]:
n_n = int(hypopt_trials_7.best_trial['misc']['vals']['n_n'][0])
m_dist = hypopt_trials_7.best_trial['misc']['vals']['m_d'][0]
r_m_dist = round(m_dist, 3)
dis_h_um = umap.UMAP(n_neighbors=n_n,
                     min_dist = m_dist,
                     n_components = 3,
                     random_state = 42).fit_transform(dis_h_2_scaled)
if hypopt_trials_7.best_trial['misc']['vals']['model'][0] == 0:
    clusterer = KMeans(init = 'random', n_clusters = 7, max_iter=500, random_state=42)
    cluster_labels = clusterer.fit_predict(dis_h_um)
    model_name = 'K-Means'
if hypopt_trials_7.best_trial['misc']['vals']['model'][0] == 1:
    clusterer = KMedoids(init = 'random', n_clusters = 7, max_iter=500, random_state=42)
    cluster_labels = clusterer.fit_predict(dis_h_um)
    model_name = 'K-Medoids'
    
fig = plt.figure(figsize = (20, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(dis_h_um[:,0], dis_h_um[:,1], dis_h_um[:,2], c = [sns.color_palette('dark',7)[x] for x in cluster_labels], s = 50)
ax.set_title(f'K가 7일때 군집 결과 \n 사용모델 : {model_name}, n_neighbor : {n_n}, min_dist : {r_m_dist}', fontsize = 20)

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
name = 'K-Means' if hypopt_trials_7.best_trial['misc']['vals']['model'][0] == 0 else 'K-Medoids'
name
fig.suptitle(f'{name} 실루엣 계수 그래프: K가 7일때', fontsize = 30)

visualizer = SilhouetteVisualizer(clusterer, colors='yellowbrick')
visualizer.fit(dis_h_um)

In [ ]:
sns.barplot(data = pd.DataFrame({'acc' : acc_c, 'cl_label' : cluster_labels}).groupby(['cl_label']).mean().reset_index(), x = 'cl_label', y = 'acc').set_title(f'군집 별 평균 사고 횟수')

In [ ]:
sns.barplot(data = pd.DataFrame(pd.DataFrame({'acc' : acc_c, 'cl_label' : cluster_labels}).cl_label.value_counts()).reset_index(), x = 'index', y = 'cl_label').set_title('군집 별 개체 갯수')

In [ ]:
res_df = pd.read_csv('분석용 데이터 진짜최종_.csv', encoding='EUC-KR').drop(columns = 'Unnamed: 2')

In [ ]:
res_df['cluster_k7'] = cluster_labels
res_df.to_csv('분석용 데이터 진짜최종_클러스터추가.csv', encoding='EUC-KR',index=False)

In [ ]:
dis_h_2.groupby('cluster_k7').mean().agg(['std','mean'])

In [ ]:
n_n = 4
m_dist = 0.01

dis_h_um_visualize = umap.UMAP(n_neighbors=n_n,
                     min_dist = m_dist,
                     n_components = 3,
                     random_state = 42).fit_transform(dis_h_2_scaled)

    
fig = plt.figure(figsize = (20, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(dis_h_um_visualize[:,0], dis_h_um_visualize[:,1], dis_h_um_visualize[:,2], s = 50, c = '#642EFE')
ax.set_title(f'UMAP 3차원 축소 결과 \n n_neighbor : {n_n}, min_dist : {m_dist}')

In [ ]:
n_n = 50
m_dist = 0.4

dis_h_um_visualize = umap.UMAP(n_neighbors=n_n,
                     min_dist = m_dist,
                     n_components = 3,
                     random_state = 42).fit_transform(dis_h_2_scaled)

    
fig = plt.figure(figsize = (20, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(dis_h_um_visualize[:,0], dis_h_um_visualize[:,1], dis_h_um_visualize[:,2], s = 50, c = '#642EFE')
ax.set_title(f'UMAP 3차원 축소 결과 \n n_neighbor : {n_n}, min_dist : {m_dist}')

In [ ]:
#Bayesian optimization results

k = []
min_dist = []
model = []
n_neighbors = []

for i in range(3,8):
    a = eval(f'hypopt_trials_{i}')
    k.append(i)
    min_dist.append(a.best_trial['misc']['vals']['m_d'][0])
    model.append('K-Means' if a.best_trial['misc']['vals']['model'][0] == 0 else 'K-Medoids')
    n_neighbors.append(a.best_trial['misc']['vals']['n_n'][0])

In [ ]:
pd.DataFrame({'K' : k, 'min_dist' : min_dist, 'model' : model, 'n_neibors' : n_neighbors})

In [ ]:
fig, ax = plt.subplots(figsize=(8,7))
name = 'K-Means' if hypopt_trials_7.best_trial['misc']['vals']['model'][0] == 0 else 'K-Medoids'
ax.set_facecolor((1.0, 1.0,1.0))
plt.rcParams['figure.facecolor'] = 'white'
visualizer = SilhouetteVisualizer(clusterer, colors='yellowbrick')
visualizer.fit(dis_h_um)

In [ ]:
cluster_labels